In [0]:
#Load data
df = spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("dbfs:/FileStore/shared_uploads/xavier211192@gmail.com/data/retail-data/all/online_retail_dataset.csv").coalesce(5)
df.cache()
df.createOrReplaceTempView('dfTable')

## Basic aggregations ##

In [0]:
##Basic Aggregation
#count #df.count() is an action df.select(count) is a transformation
from pyspark.sql.functions import count,countDistinct,approx_count_distinct
df.select(count("StockCode")).show()

#count distinct
df.select(countDistinct("StockCode")).show()

#count approx distinct
df.select(approx_count_distinct("StockCode",0.05)).show()

+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4070|
+-------------------------+

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3804|
+--------------------------------+

In [0]:
from pyspark.sql.functions import first,last,min,max
#first and last
df.select(first("StockCode"),last("StockCode")).show()
#min and max
df.select(min("Quantity"),max("Quantity")).show()

+----------------+---------------+
first(StockCode)|last(StockCode)|
+----------------+---------------+
 85123A| 22138|
+----------------+---------------+

+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

In [0]:
from pyspark.sql.functions import sum,sumDistinct
#sum
df.select(sum("Quantity")).show()
#sum distinct
df.select(sumDistinct("Quantity")).show()

+-------------+
sum(Quantity)|
+-------------+
 5176450|
+-------------+

+----------------------+
sum(DISTINCT Quantity)|
+----------------------+
 29310|
+----------------------+

In [0]:
from pyspark.sql.functions import sum, count, avg, expr
#avg and mean are the same: syntax is different.
df.select(
    count("Quantity").alias("total"),
    sum("Quantity").alias("sum"),
    avg("Quantity").alias("avg"),
    expr("mean(Quantity)").alias("mean"))\
    .selectExpr(
    "total/sum","avg","mean").show()

+-------------------+----------------+----------------+
 (total / sum)| avg| mean|
+-------------------+----------------+----------------+
0.10468738227936134|9.55224954743324|9.55224954743324|
+-------------------+----------------+----------------+

In [0]:
#sample and population variance and std deviation
from pyspark.sql.functions import var_pop,stddev_pop
from pyspark.sql.functions import var_samp,stddev_samp

df.select(var_pop("Quantity"),var_samp("Quantity"),
         stddev_pop("Quantity"),stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
 var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
47559.303646609056|47559.391409298754| 218.08095663447796| 218.08115785023418|
+------------------+------------------+--------------------+---------------------+

In [0]:
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"),kurtosis("Quantity")).show()

+-------------------+------------------+
 skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
-0.2640755761052562|119768.05495536952|
+-------------------+------------------+

In [0]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo","Quantity"),
         covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
 4.912186085635685E-4| 1052.7280543902734| 1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+

In [0]:
from pyspark.sql.functions import collect_list,collect_set
#All values
df.select(collect_list("Country")).show()
#unique values
df.select(collect_set("Country")).show()

+---------------------+
collect_list(Country)|
+---------------------+
 [United Kingdom, ...|
+---------------------+

+--------------------+
collect_set(Country)|
+--------------------+
[Portugal, Italy,...|
+--------------------+

## Groupings ##

In [0]:
#Simple grouping
df.groupBy("InvoiceNo","CustomerId").count().show()

+---------+----------+-----+
InvoiceNo|CustomerId|count|
+---------+----------+-----+
 536846| 14573| 76|
 537026| 12395| 12|
 537883| 14437| 5|
 538068| 17978| 12|
 538279| 14952| 7|
 538800| 16458| 10|
 538942| 17346| 12|
 C539947| 13854| 1|
 540096| 13253| 16|
 540530| 14755| 27|
 541225| 14099| 19|
 541978| 13551| 4|
 542093| 17677| 16|
 536596| null| 6|
 537252| null| 1|
 538041| null| 1|
 537159| 14527| 28|
 537213| 12748| 6|
 538191| 15061| 16|
 C539301| 13496| 1|
+---------+----------+-----+
only showing top 20 rows

In [0]:
#Grouping with Expressions
from pyspark.sql.functions import count
df.groupBy("InvoiceNo").agg(count("Quantity").alias("cnt_quantity"), expr("count(Quantity)")).show()

+---------+------------+---------------+
InvoiceNo|cnt_quantity|count(Quantity)|
+---------+------------+---------------+
 536596| 6| 6|
 536938| 14| 14|
 537252| 1| 1|
 537691| 20| 20|
 538041| 1| 1|
 538184| 26| 26|
 538517| 53| 53|
 538879| 19| 19|
 539275| 6| 6|
 539630| 12| 12|
 540499| 24| 24|
 540540| 22| 22|
 C540850| 1| 1|
 540976| 48| 48|
 541432| 4| 4|
 541518| 101| 101|
 541783| 35| 35|
 542026| 9| 9|
 542375| 6| 6|
 536597| 28| 28|
+---------+------------+---------------+
only showing top 20 rows

In [0]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
InvoiceNo| avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
 536596| 1.5| 1.1180339887498947|
 536938|33.142857142857146| 20.698023172885524|
 537252| 31.0| 0.0|
 537691| 8.15| 5.597097462078001|
 538041| 30.0| 0.0|
 538184|12.076923076923077| 8.142590198943392|
 538517|3.0377358490566038| 2.3946659604837897|
 538879|21.157894736842106| 11.811070444356483|
 539275| 26.0| 12.806248474865697|
 539630|20.333333333333332| 10.225241100118645|
 540499| 3.75| 2.6653642652865788|
 540540|2.1363636363636362| 1.0572457590557278|
 C540850| -1.0| 0.0|
 540976|10.520833333333334| 6.496760677872902|
 541432| 12.25| 10.825317547305483|
 541518| 23.10891089108911| 20.550782784878713|
 541783|11.314285714285715| 8.467657556242811|
 542026| 7.666666666666667| 4.853406592853679|
 542375| 8.0| 3.4641016151377544|
 536597|2.5357142857142856| 2.7448932175059566|
+---------+------------------+--------------------+
only showing top 20 rows

## Window functions ##

In [0]:
#Add a new date column
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.functions import col, to_date, desc, max, dense_rank
from pyspark.sql.window import Window
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))

#1 define a window - use all preceding rows up and until current row
windowSpec = Window\
            .partitionBy("CustomerID","date")\
            .orderBy(desc("Quantity"))\
            .rowsBetween(Window.unboundedPreceding, Window.currentRow)
#2 define aggregation maxPurchaseQuantity
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)
print(maxPurchaseQuantity)

#2.1 purchaseRank
purchaseDenseRank = dense_rank().over(windowSpec)
print(purchaseDenseRank)

Column<b'max(Quantity) OVER (PARTITION BY CustomerID, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)'>
Column<b'DENSE_RANK() OVER (PARTITION BY CustomerID, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)'>

In [0]:
# 3 Perform a Select 
dfWithDate.where("CustomerId IS NOT NULL AND ").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()


+----------+----------+--------+-----------------+-------------------+
CustomerId| date|Quantity|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+-----------------+-------------------+
 12346|2011-01-18| 74215| 1| 74215|
 12346|2011-01-18| -74215| 2| 74215|
 12347|2010-12-07| 36| 1| 36|
 12347|2010-12-07| 30| 2| 36|
 12347|2010-12-07| 24| 3| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 12| 4| 36|
 12347|2010-12-07| 6| 5| 36|
 12347|2010-12-07| 6| 5| 36|
+----------+----------+--------+-----------------+-------------------+
only showing top 20 rows

In [0]:
# 1 define window spec
windowSpec = Window.partitionBy("CustomerId","date").orderBy("Quantity").rowsBetween(Window.unboundedPreceding, Window.currentRow)
# 2 rank and min purchase quantity
minPurchaseQuantity = min("Quantity").over(windowSpec)
reverseRank = dense_rank().over(windowSpec)

dfWithDate.where("CustomerId is NOT NULL").\
                 select(
                   col("CustomerId"),
                   col("date"),
                   col("Quantity"),
                   reverseRank.alias("reverseRank"),
                   minPurchaseQuantity.alias("minPurchase")
                 ).orderBy("CustomerId").show()

+----------+----------+--------+-----------+-----------+
CustomerId| date|Quantity|reverseRank|minPurchase|
+----------+----------+--------+-----------+-----------+
 12346|2011-01-18| -74215| 1| -74215|
 12346|2011-01-18| 74215| 2| -74215|
 12347|2010-12-07| 3| 1| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 4| 2| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 6| 3| 3|
 12347|2010-12-07| 12| 4| 3|
 12347|2010-12-07| 12| 4| 3|
 12347|2010-12-07| 12| 4| 3|
+----------+----------+--------+-----------+-----------+
only showing top 20 rows

### Grouping Sets ###

In [0]:
#drop nulls
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView('dfNoNull')

In [0]:
from pyspark.sql.functions import  sum
dfNoNull.groupBy("CustomerId","StockCode").sum("Quantity").show()

In [0]:
%sql
--#Total quantity of all stock codes and customers
Select CustomerId,StockCode, SUM(QUANTITY) from dfNoNull
group by CustomerId,StockCode order by CustomerID desc, StockCode desc

CustomerId,StockCode,sum(QUANTITY)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


In [0]:
%sql
--Using Grouping sets achieve the same 
Select CustomerId,StockCode, SUM(QUANTITY) from dfNoNull
group by CustomerId,StockCode GROUPING SETS((customerId,stockCode))
order by CustomerID desc, StockCode desc

CustomerId,StockCode,sum(QUANTITY)
18287,85173,48
18287,85040A,48
18287,85039B,120
18287,85039A,96
18287,84920,4
18287,84584,6
18287,84507C,6
18287,72351B,24
18287,72351A,24
18287,72349B,60


In [0]:
#Rollup over date and country
rolledUpDF = dfNoNull.rollup("Date","Country").sum("Quantity").orderBy("Date")

In [0]:
display(rolledUpDF)
#where both are nulls is the grandtotal

Country,date,sum(Quantity)
Saudi Arabia,null,75
EIRE,null,142637
Bahrain,null,260
United Arab Emirates,null,982
Israel,null,4353
Czech Republic,null,592
Austria,null,4827
USA,null,1034
European Community,null,497
Belgium,null,23152


In [0]:
rolledUpDF.where("Country is NULL").show()

+----------+-------+-------------+
 Date|Country|sum(Quantity)|
+----------+-------+-------------+
 null| null| 5176450|
2010-12-01| null| 26814|
2010-12-02| null| 21023|
2010-12-03| null| 14830|
2010-12-05| null| 16395|
2010-12-06| null| 21419|
2010-12-07| null| 24995|
2010-12-08| null| 22741|
2010-12-09| null| 18431|
2010-12-10| null| 20297|
2010-12-12| null| 10565|
2010-12-13| null| 17623|
2010-12-14| null| 20098|
2010-12-15| null| 18229|
2010-12-16| null| 29632|
2010-12-17| null| 16069|
2010-12-19| null| 3795|
2010-12-20| null| 14965|
2010-12-21| null| 15467|
2010-12-22| null| 3192|
+----------+-------+-------------+
only showing top 20 rows

In [0]:
#where one of the column is null is subtotal
rolledUpDF.where("Country is NULL").show()
rolledUpDF.where("Date is NULL").show()

+----------+-------+-------------+
 Date|Country|sum(Quantity)|
+----------+-------+-------------+
 null| null| 5176450|
2010-12-01| null| 26814|
2010-12-02| null| 21023|
2010-12-03| null| 14830|
2010-12-05| null| 16395|
2010-12-06| null| 21419|
2010-12-07| null| 24995|
2010-12-08| null| 22741|
2010-12-09| null| 18431|
2010-12-10| null| 20297|
2010-12-12| null| 10565|
2010-12-13| null| 17623|
2010-12-14| null| 20098|
2010-12-15| null| 18229|
2010-12-16| null| 29632|
2010-12-17| null| 16069|
2010-12-19| null| 3795|
2010-12-20| null| 14965|
2010-12-21| null| 15467|
2010-12-22| null| 3192|
+----------+-------+-------------+
only showing top 20 rows

+----+-------+-------------+
Date|Country|sum(Quantity)|
+----+-------+-------------+
null| null| 5176450|
+----+-------+-------------+

In [0]:
#Cube takes the rollup to a level deeper.
dfNoNull.cube("Date","Country").sum("Quantity").orderBy("Date").show()

+----+---------------+-------------+
Date| Country|sum(Quantity)|
+----+---------------+-------------+
null| Lithuania| 652|
null| Belgium| 23152|
null| Australia| 83653|
null| Denmark| 8188|
null| Spain| 26824|
null|Channel Islands| 9479|
null| Singapore| 5234|
null| Lebanon| 386|
null| Cyprus| 6317|
null| Hong Kong| 4769|
null| Finland| 10666|
null| Germany| 117448|
null| null| 5176450|
null| Norway| 19247|
null| Portugal| 16180|
null| Japan| 25218|
null| Italy| 7999|
null| Greece| 1556|
null| Iceland| 2458|
null| Netherlands| 200128|
+----+---------------+-------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import grouping_id,sum,desc
dfNoNull.cube("CustomerId","StockCode")\
        .agg(grouping_id(), sum(col("Quantity")))\
        .orderBy(desc("grouping_id()")).show()

+----------+---------+-------------+-------------+
CustomerId|StockCode|grouping_id()|sum(Quantity)|
+----------+---------+-------------+-------------+
 null| null| 3| 5176450|
 null| 72127| 2| 140|
 null| 85025B| 2| 330|
 null| 84799| 2| -182|
 null| 90059D| 2| 19|
 null| 22533| 2| 1743|
 null| 47343A| 2| 143|
 null| 22627| 2| 841|
 null| 22952| 2| 15767|
 null| 90209B| 2| 186|
 null| 90093| 2| 52|
 null| 37446| 2| 1292|
 null| 37503| 2| -1|
 null| 21827| 2| -36|
 null| 22714| 2| 3475|
 null| 84922| 2| 9|
 null| 22224| 2| 831|
 null| 22825| 2| 71|
 null| 22927| 2| 372|
 null| 22560| 2| 8167|
+----------+---------+-------------+-------------+
only showing top 20 rows

In [0]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
display(pivoted)

date,Australia_sum(CAST(Quantity AS BIGINT)),Australia_sum(UnitPrice),Australia_sum(CAST(CustomerID AS BIGINT)),Austria_sum(CAST(Quantity AS BIGINT)),Austria_sum(UnitPrice),Austria_sum(CAST(CustomerID AS BIGINT)),Bahrain_sum(CAST(Quantity AS BIGINT)),Bahrain_sum(UnitPrice),Bahrain_sum(CAST(CustomerID AS BIGINT)),Belgium_sum(CAST(Quantity AS BIGINT)),Belgium_sum(UnitPrice),Belgium_sum(CAST(CustomerID AS BIGINT)),Brazil_sum(CAST(Quantity AS BIGINT)),Brazil_sum(UnitPrice),Brazil_sum(CAST(CustomerID AS BIGINT)),Canada_sum(CAST(Quantity AS BIGINT)),Canada_sum(UnitPrice),Canada_sum(CAST(CustomerID AS BIGINT)),Channel Islands_sum(CAST(Quantity AS BIGINT)),Channel Islands_sum(UnitPrice),Channel Islands_sum(CAST(CustomerID AS BIGINT)),Cyprus_sum(CAST(Quantity AS BIGINT)),Cyprus_sum(UnitPrice),Cyprus_sum(CAST(CustomerID AS BIGINT)),Czech Republic_sum(CAST(Quantity AS BIGINT)),Czech Republic_sum(UnitPrice),Czech Republic_sum(CAST(CustomerID AS BIGINT)),Denmark_sum(CAST(Quantity AS BIGINT)),Denmark_sum(UnitPrice),Denmark_sum(CAST(CustomerID AS BIGINT)),EIRE_sum(CAST(Quantity AS BIGINT)),EIRE_sum(UnitPrice),EIRE_sum(CAST(CustomerID AS BIGINT)),European Community_sum(CAST(Quantity AS BIGINT)),European Community_sum(UnitPrice),European Community_sum(CAST(CustomerID AS BIGINT)),Finland_sum(CAST(Quantity AS BIGINT)),Finland_sum(UnitPrice),Finland_sum(CAST(CustomerID AS BIGINT)),France_sum(CAST(Quantity AS BIGINT)),France_sum(UnitPrice),France_sum(CAST(CustomerID AS BIGINT)),Germany_sum(CAST(Quantity AS BIGINT)),Germany_sum(UnitPrice),Germany_sum(CAST(CustomerID AS BIGINT)),Greece_sum(CAST(Quantity AS BIGINT)),Greece_sum(UnitPrice),Greece_sum(CAST(CustomerID AS BIGINT)),Hong Kong_sum(CAST(Quantity AS BIGINT)),Hong Kong_sum(UnitPrice),Hong Kong_sum(CAST(CustomerID AS BIGINT)),Iceland_sum(CAST(Quantity AS BIGINT)),Iceland_sum(UnitPrice),Iceland_sum(CAST(CustomerID AS BIGINT)),Israel_sum(CAST(Quantity AS BIGINT)),Israel_sum(UnitPrice),Israel_sum(CAST(CustomerID AS BIGINT)),Italy_sum(CAST(Quantity AS BIGINT)),Italy_sum(UnitPrice),Italy_sum(CAST(CustomerID AS BIGINT)),Japan_sum(CAST(Quantity AS BIGINT)),Japan_sum(UnitPrice),Japan_sum(CAST(CustomerID AS BIGINT)),Lebanon_sum(CAST(Quantity AS BIGINT)),Lebanon_sum(UnitPrice),Lebanon_sum(CAST(CustomerID AS BIGINT)),Lithuania_sum(CAST(Quantity AS BIGINT)),Lithuania_sum(UnitPrice),Lithuania_sum(CAST(CustomerID AS BIGINT)),Malta_sum(CAST(Quantity AS BIGINT)),Malta_sum(UnitPrice),Malta_sum(CAST(CustomerID AS BIGINT)),Netherlands_sum(CAST(Quantity AS BIGINT)),Netherlands_sum(UnitPrice),Netherlands_sum(CAST(CustomerID AS BIGINT)),Norway_sum(CAST(Quantity AS BIGINT)),Norway_sum(UnitPrice),Norway_sum(CAST(CustomerID AS BIGINT)),Poland_sum(CAST(Quantity AS BIGINT)),Poland_sum(UnitPrice),Poland_sum(CAST(CustomerID AS BIGINT)),Portugal_sum(CAST(Quantity AS BIGINT)),Portugal_sum(UnitPrice),Portugal_sum(CAST(CustomerID AS BIGINT)),RSA_sum(CAST(Quantity AS BIGINT)),RSA_sum(UnitPrice),RSA_sum(CAST(CustomerID AS BIGINT)),Saudi Arabia_sum(CAST(Quantity AS BIGINT)),Saudi Arabia_sum(UnitPrice),Saudi Arabia_sum(CAST(CustomerID AS BIGINT)),Singapore_sum(CAST(Quantity AS BIGINT)),Singapore_sum(UnitPrice),Singapore_sum(CAST(CustomerID AS BIGINT)),Spain_sum(CAST(Quantity AS BIGINT)),Spain_sum(UnitPrice),Spain_sum(CAST(CustomerID AS BIGINT)),Sweden_sum(CAST(Quantity AS BIGINT)),Sweden_sum(UnitPrice),Sweden_sum(CAST(CustomerID AS BIGINT)),Switzerland_sum(CAST(Quantity AS BIGINT)),Switzerland_sum(UnitPrice),Switzerland_sum(CAST(CustomerID AS BIGINT)),USA_sum(CAST(Quantity AS BIGINT)),USA_sum(UnitPrice),USA_sum(CAST(CustomerID AS BIGINT)),United Arab Emirates_sum(CAST(Quantity AS BIGINT)),United Arab Emirates_sum(UnitPrice),United Arab Emirates_sum(CAST(CustomerID AS BIGINT)),United Kingdom_sum(CAST(Quantity AS BIGINT)),United Kingdom_sum(UnitPrice),United Kingdom_sum(CAST(CustomerID AS BIGINT)),Unspecified_sum(CAST(Quantity AS BIGINT)),Unspecified_sum(UnitPrice),Unspecified_sum(CAST(CustomerID AS BIGINT))
2011-05-06,null,null,null

In [0]:
from pyspark.sql.functions import collect_set,collect_list
from pyspark.sql.window import Window

window = Window\
                   .partitionBy("date","Country")\
                   .orderBy(desc("Quantity"))\
                   .rowsBetween(Window.unboundedPreceding, Window.currentRow)

max_q = max(col("Quantity")).over(window)

wind_df = dfWithDate.select(col("Date"),col("Country"),col("Quantity"),max_q.alias("max_q"))

In [0]:
%sql

Select Date,Country, sum(Quantity) from dfNoNull
group by Date,Country grouping sets(Date,Country,())
order by Date

Date,Country,sum(Quantity)
null,Austria,4827
null,null,5176450
null,Switzerland,30325
null,Poland,3653
null,Lebanon,386
null,Australia,83653
null,Sweden,35637
null,Spain,26824
null,Bahrain,260
null,Portugal,16180
